# AIAP Batch 15 Technical Assessment
*written by Billy Lim Jun Ming*

## Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [2]:
# read pre data
con_pre = sqlite3.connect("data/cruise_pre.db")
cur_pre = con_pre.execute("SELECT * FROM cruise_pre")

# print head of database
res = con_pre.execute("SELECT * FROM cruise_pre LIMIT 5")
print(pd.DataFrame(data=res, columns=[i[0] for i in cur_pre.description]))

   index  Gender Date of Birth         Source of Traffic  \
0      0    None    05/10/1973  Direct - Company Website   
1      1  Female          None   Indirect - Social Media   
2      2  Female    22/07/1998  Indirect - Search Engine   
3      3  Female    01/05/1970  Direct - Company Website   
4      4    Male    07/01/1960  Direct - Company Website   

   Onboard Wifi Service  Embarkation/Disembarkation time convenient  \
0    A little important                                         3.0   
1  Not at all important                                         4.0   
2                  None                                         3.0   
3        Very important                                         4.0   
4    Somewhat important                                         4.0   

   Ease of Online booking  Gate location          Logging  \
0                     5.0            3.0  01/01/2023 0:00   
1                     1.0            NaN  01/01/2023 0:01   
2                     0.0    

In [3]:
# read post_data
con_post = sqlite3.connect("data/cruise_post.db")
cur_post = con_post.execute("SELECT * FROM cruise_post")

# print head of database
res = con_post.execute("SELECT * FROM cruise_post LIMIT 5")
print(pd.DataFrame(data=res, columns=[i[0] for i in cur_post.description]))

   index Cruise Name Ticket Type Cruise Distance  Ext_Intcode  WiFi  Dining  \
0      0   Blastoise        None         3567 KM  LB446RWOOZI   1.0       1   
1      1   Blastoise      Deluxe          672 KM  LB138HKBECM   NaN       0   
2      2      IAPRAS      Deluxe         1167 KM  BL713UHBAAN   NaN       0   
3      3      Lapras      Deluxe          280 KM  LB243DMKCFL   NaN       0   
4      4      Lapras    Standard      1145 Miles  LB218CFLOBS   NaN       1   

   Entertainment  
0            1.0  
1            1.0  
2            0.0  
3            1.0  
4            NaN  
